<img src="https://industrial.uniandes.edu.co/sites/default/files/imagenes/uniandeslogo.png" alt="Universidad de los Andes" style="float: right; width: 300px; height: auto;">

# commodities_extraction_yfinance

Editor: Juan Diego Heredia - jd.heredia@uniandes.edu.co

Febrero 2026

### Importar librerías

La siguiente celda marca el inicio de un nuevo bloque de trabajo independiente:

**Librerías utilizadas:**
- `pandas` y `numpy`: para manipulación y análisis de datos estructurados.
- `warnings`: para suprimir advertencias irrelevantes durante la ejecución.
- `yfinance`: para descargar datos históricos de Yahoo Finance.
- `yaml`: para leer archivos de configuración YAML.
- `pathlib.Path`: para manejo de rutas multiplataforma.

In [1]:
import pandas as pd
import numpy as np
import warnings
#!pip install yfinance
import yfinance as yf

import yaml
from pathlib import Path

### Configuración de rutas

Carga los directorios principales desde el archivo de configuración `paths.yml` para mantener rutas organizadas y portables entre sistemas.

In [2]:
# Carga directorios desde archivo de configuración
with open('paths.yml', 'r') as file:
    paths = yaml.safe_load(file)

# Crea objetos Path para cada directorio
temp = Path(paths['data']['temp'])

### Configuración de opciones

Establece configuraciones que aplican a todo el notebook:
- Supresión de advertencias

In [3]:
warnings.filterwarnings('ignore')

### Parámetros de extracción

Define las variables parametrizadas para la extracción de datos desde Yahoo Finance, incluyendo fechas, tickers de futuros y configuración de columnas.

In [4]:
# Configuración general
fecha_inicio = '2004-01-01'
fecha_fin = '2025-12-31'

# Tickers de Yahoo Finance
yfinance_tickers = {
    'cafe_arabica': 'KC=F',
    'petroleo_crudo': 'CL=F',
    'oro': 'GC=F'
}

# Columnas finales
columnas_finales = [
    'quarter',
    'petroleo_crudo_mean',
    'petroleo_crudo_median',
    'cafe_arabica_mean',
    'cafe_arabica_median',
    'oro_mean',
    'oro_median'
]

### Descarga de datos históricos

Descarga los datos históricos de precios para cada commodity desde Yahoo Finance. Extrae la columna 'Adj Close' como precio de referencia y almacena cada serie en un diccionario temporal.

In [5]:
# Descarga datos históricos
series_descargadas = {}

for nombre, ticker in yfinance_tickers.items():
    datos = yf.download(ticker, start=fecha_inicio, end=fecha_fin, progress=False)
    series_descargadas[nombre] = datos['Close']
    print(f"{nombre}: {len(datos)} observaciones descargadas")

cafe_arabica: 5526 observaciones descargadas
petroleo_crudo: 5530 observaciones descargadas
oro: 5526 observaciones descargadas


### Consolidación de series

Combina todas las series descargadas en un único dataframe con fechas como índice. Las columnas corresponden a cada commodity con sus precios ajustados de cierre.

In [6]:
# Concatena y aplana los nombres de columnas multinivel
df_diario = pd.concat(series_descargadas, axis=1)
df_diario.columns = df_diario.columns.get_level_values(0)  # Toma solo el primer nivel
df_diario.index.name = 'fecha'

### Agregación trimestral

Resamplea los datos diarios a frecuencia trimestral, calculando el promedio y la mediana de precios para cada commodity en cada trimestre.

In [7]:
# Calcula promedio trimestral
df_mean = df_diario.resample('Q').mean()
df_mean.columns = [f"{col}_mean" for col in df_mean.columns]

# Calcula mediana trimestral
df_median = df_diario.resample('Q').median()
df_median.columns = [f"{col}_median" for col in df_median.columns]

# Combina mean y median
df_trimestral = pd.concat([df_mean, df_median], axis=1)

### Creación de columna quarter

Genera la columna 'quarter' en formato "YYYYQ#" a partir del índice de fechas trimestrales. Reorganiza las columnas según el orden establecido en los parámetros.

In [8]:
# Crea columna quarter en formato YYYYQ#
df_trimestral['quarter'] = df_trimestral.index.to_period('Q').astype(str)
df_trimestral = df_trimestral.reset_index(drop=True)

# Reordena columnas según especificación
df_trimestral = df_trimestral[columnas_finales]

### Sanity checks

Ejecuta verificaciones de calidad de datos para validar la integridad del dataframe resultante: total de registros, rango de fechas, valores nulos, valores negativos y duplicados.

In [9]:
# Sanity checks
print("=== SANITY CHECKS - YAHOO FINANCE ===")
print(f"Total records: {len(df_trimestral)}")
print(f"Date range: {df_trimestral['quarter'].iloc[0]} to {df_trimestral['quarter'].iloc[-1]}")

columnas_numericas = [col for col in df_trimestral.columns if col != 'quarter']
null_count = df_trimestral[columnas_numericas].isnull().sum().sum()
print(f"Null values: {null_count}")

negative_count = (df_trimestral[columnas_numericas] < 0).sum().sum()
print(f"Negative values: {negative_count}")

duplicates = df_trimestral['quarter'].duplicated().sum()
print(f"Duplicates: {duplicates}")

=== SANITY CHECKS - YAHOO FINANCE ===
Total records: 88
Date range: 2004Q1 to 2025Q4
Null values: 0
Negative values: 0
Duplicates: 0


In [10]:
df_trimestral.head()

,quarter,petroleo_crudo_mean,petroleo_crudo_median,cafe_arabica_mean,cafe_arabica_median,oro_mean,oro_median
0,2004Q1,35.250164,35.160000,73.244262,73.699997,408.634426,407.899994
1,2004Q2,38.283710,37.790001,73.577419,72.200001,393.458063,391.949997
2,2004Q3,43.892031,43.844999,70.982031,69.500000,402.154686,403.699997
3,2004Q4,48.271935,48.840000,86.406452,83.750000,434.058064,435.750000
4,2005Q1,50.030984,48.779999,114.850000,115.099998,427.866667,426.449997


### Guardado de datos

Exporta el dataframe trimestral en formato parquet a la carpeta temporal de commodities. Verifica la existencia de la carpeta destino y la crea si es necesario.

In [11]:
# Verifica existencia de carpeta y crea si no existe
output_path = temp / 'commodities'
output_path.mkdir(parents=True, exist_ok=True)

# Guarda archivo
df_trimestral.to_parquet(output_path / 'commodities.parquet', index=False)
print(f"Archivo guardado en: {output_path / 'commodities.parquet'}")

Archivo guardado en: /Users/juandiegoheredianino/Library/CloudStorage/OneDrive-Universidaddelosandes/early-warning-atypical-violence-forecast/data/temp/commodities/commodities.parquet
